In [3]:
import requests
import json
from datetime import datetime

# Definer API-endepunkt og parametere
url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
params = {
    "lat": 51.5,  # Breddegrad for London
    "lon": 0       # Lengdegrad for London
}

# User-Agent er påkrevd av Yr
headers = {
    "User-Agent": "Miljoedataanalyseapplikasjon/0.1 (fredbm@stud.ntnu.no)"
}

# Gjør API-kallet
response = requests.get(url, headers=headers, params=params)

# Sjekk om forespørselen var vellykket
if response.status_code == 200:
    data = response.json()

    # Hent ut den nyeste værprognosen
    forecast = data["properties"]["timeseries"][0]
    time = forecast["time"]
    details = forecast["data"]["instant"]["details"]

    # Formater dataene
    temperature = details["air_temperature"]
    wind_speed = details["wind_speed"]
    precipitation = forecast["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount", 0)

    # Konverter tidspunkt til leselig format
    time_formatted = datetime.fromisoformat(time.replace("Z", "+00:00")).strftime("%Y-%m-%d %H:%M:%S")

    # Print værdata på en ryddig måte
    print("=" * 40)
    print(f"📍 Værmelding for London (51.5°N, 0°E)")
    print(f"🕒 Tidspunkt: {time_formatted} UTC")
    print(f"🌡 Temperatur: {temperature}°C")
    print(f"💨 Vindhastighet: {wind_speed} m/s")
    print(f"🌧 Nedbør (neste time): {precipitation} mm")
    print("=" * 40)

else:
    print(f"Feil: {response.status_code} - {response.text}")


📍 Værmelding for London (51.5°N, 0°E)
🕒 Tidspunkt: 2025-02-26 12:00:00 UTC
🌡 Temperatur: 7.2°C
💨 Vindhastighet: 8.2 m/s
🌧 Nedbør (neste time): 1.8 mm
